<a href="https://colab.research.google.com/github/kateburovova/dehumanization/blob/mainbranch/finetune_RuBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [3]:
df_labels = pd.read_csv('/content/final_labels.csv', index_col=[0])

In [4]:
df_mention = df_labels[['text', 'Mention']]
df_emotion = df_labels[['text', 'Emotion']]

In [5]:
mention_label_mapping = {'так': 1, 'ні': 0, 'не можу визначитись із відповіддю': -1}
df_mention['labels'] = df_mention['Mention'].map(mention_label_mapping)

<ipython-input-5-1554e77b6fff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mention['labels'] = df_mention['Mention'].map(mention_label_mapping)


In [6]:
# dropping lines where labelers are unsure
df_mention = df_mention[df_mention['labels']!=-1]
len(df_mention)

3518

In [7]:
emotion_label_mapping = {'так, присутня негативна': 1,
                         'ні, оцінка не присутня': 0,
                         'не можу визначитись з правильною відповіддю': -1,
                         'так, присутня позитивна': 2}
df_emotion['labels'] = df_emotion['Emotion'].map(emotion_label_mapping)

<ipython-input-7-b63af5414c05>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emotion['labels'] = df_emotion['Emotion'].map(emotion_label_mapping)


In [8]:
df_emotion = df_emotion[df_emotion.labels!=-1] #dropping those regarding which labelers are unsure

In [9]:
len(df_emotion)


3495

In [11]:
pip install wandb


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=7581dedc8ce04059633a0a2401e80dd8921dac07016eb5d5703fc8dbc7011b4e
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [13]:
import wandb

In [15]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.0 MB/s eta 0:00:00


In [16]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments

In [17]:
train_data, val_data = train_test_split(df_mention, test_size=0.2, random_state=42)
max_length = 512

In [18]:
tokenizer = BertTokenizerFast.from_pretrained("DeepPavlov/rubert-base-cased")
train_encodings = tokenizer(train_data["text"].tolist(), padding=True, truncation=True, max_length=max_length)
val_encodings = tokenizer(val_data["text"].tolist(), padding=True, truncation=True, max_length=max_length)

In [19]:
class BinaryClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = BinaryClassificationDataset(train_encodings, train_data["labels"].tolist())
val_dataset = BinaryClassificationDataset(val_encodings, val_data["labels"].tolist())

In [20]:
model = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=2)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [34]:
# Load the pre-trained model
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    weight_decay=0.01,
    metric_for_best_model="f1_score",
    report_to="wandb",
)

def compute_accuracy(predictions, labels):
    preds = predictions.argmax(-1)
    return {"accuracy": (preds == labels).mean()}

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     return compute_accuracy(predictions, labels)

def compute_f1_score(predictions, labels):
    preds = predictions.argmax(-1)
    return f1_score(labels, preds, average="weighted")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {"f1_score": compute_f1_score(predictions, labels)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [35]:
wandb.init(project="binary_sentence_classification", name="rubert_mention_finetuning_colab")


In [36]:
# Start training
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score
1,0.098900,0.334032,0.952794
2,0.006200,0.279004,0.963240
3,0.000300,0.315757,0.952665


TrainOutput(global_step=1056, training_loss=0.02957363521369244, metrics={'train_runtime': 226.7908, 'train_samples_per_second': 37.224, 'train_steps_per_second': 4.656, 'total_flos': 1956550335422760.0, 'train_loss': 0.02957363521369244, 'epoch': 3.0})

In [33]:
wandb.finish()

eval/f1_score,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,▁▁▄▁▁▁▁▁▁▃▄▁█▁▃▁▁▃▃▁▂▄▃▁▆▁▁▄▁▅▁▅▁▁▅
eval/f1_score,0.95604
eval/loss,0.23262
